# import required libraries 

In [2]:
import tensorflow as tf
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt
import os
import numpy as np
from tensorflow.keras.preprocessing import image
import warnings
warnings.filterwarnings('ignore')

2025-04-15 12:52:06.272072: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744721526.442072      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744721526.497724      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# 1. Parameters

In [3]:
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS = 10
DATA_DIR = "/kaggle/input/eye-disease-image-dataset"

# 2. Load Training and Validation Datasets

In [4]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    os.path.join(DATA_DIR, '/kaggle/input/eye-disease-image-dataset/Augmented Dataset/Augmented Dataset'),
    seed=123,
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    os.path.join(DATA_DIR, '/kaggle/input/eye-disease-image-dataset/Original Dataset/Original Dataset'),
    seed=123,
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE
)

Found 16242 files belonging to 10 classes.


I0000 00:00:1744721554.094501      31 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


Found 5335 files belonging to 10 classes.


# 3. Get class names

In [5]:
class_names = train_ds.class_names
num_classes = len(class_names)
print("Classes:", class_names)

Classes: ['Central Serous Chorioretinopathy-Color Fundus', 'Diabetic Retinopathy', 'Disc Edema', 'Glaucoma', 'Healthy', 'Macular Scar', 'Myopia', 'Pterygium', 'Retinal Detachment', 'Retinitis Pigmentosa']


# 4. Improve performance with prefetching

In [6]:
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.prefetch(buffer_size=AUTOTUNE)

# 5. Build the Model (using MobileNetV2 as base)

In [7]:
base_model = tf.keras.applications.MobileNetV2(
    input_shape=IMAGE_SIZE + (3,),
    include_top=False,
    weights='imagenet'
)
base_model.trainable = False  # freeze base model

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dropout(0.2),
    layers.Dense(num_classes, activation='softmax')
])

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


# 6. Compile the Model

In [8]:
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# 7. Train the Model

In [ ]:
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS
)

Epoch 1/10


I0000 00:00:1744721571.153662      98 service.cc:148] XLA service 0x781bdc1424e0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1744721571.154337      98 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1744721572.274271      98 cuda_dnn.cc:529] Loaded cuDNN version 90300


  4/508 ━━━━━━━━━━━━━━━━━━━━ 10s 21ms/step - accuracy: 0.1543 - loss: 2.6070   

I0000 00:00:1744721576.147611      98 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


508/508 ━━━━━━━━━━━━━━━━━━━━ 136s 246ms/step - accuracy: 0.3043 - loss: 1.9238 - val_accuracy: 0.4255 - val_loss: 1.5276
Epoch 2/10
508/508 ━━━━━━━━━━━━━━━━━━━━ 86s 170ms/step - accuracy: 0.4099 - loss: 1.5971 - val_accuracy: 0.4875 - val_loss: 1.4137
Epoch 3/10
508/508 ━━━━━━━━━━━━━━━━━━━━ 87s 170ms/step - accuracy: 0.4536 - loss: 1.4895 - val_accuracy: 0.4620 - val_loss: 1.4194
Epoch 4/10
508/508 ━━━━━━━━━━━━━━━━━━━━ 87s 171ms/step - accuracy: 0.4653 - loss: 1.4415 - val_accuracy: 0.5046 - val_loss: 1.3485
Epoch 5/10
508/508 ━━━━━━━━━━━━━━━━━━━━ 103s 203ms/step - accuracy: 0.4769 - loss: 1.4146 - val_accuracy: 0.4664 - val_loss: 1.3992
Epoch 6/10
508/508 ━━━━━━━━━━━━━━━━━━━━ 104s 205ms/step - accuracy: 0.4843 - loss: 1.3991 - val_accuracy: 0.5038 - val_loss: 1.3464
Epoch 7/10
508/508 ━━━━━━━━━━━━━━━━━━━━ 87s 171ms/step - accuracy: 0.4907 - loss: 1.3800 - val_accuracy: 0.4793 - val_loss: 1.3743
Epoch 8/10
508/508 ━━━━━━━━━━━━━━━━━━━━ 86s 169ms/step - accuracy: 0.4963 - loss: 1.3648 - 

# 8. Plot Accuracy and Loss

In [ ]:
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Val Accuracy')
plt.legend()
plt.title("Accuracy")

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.legend()
plt.title("Loss")

plt.show()

# 9. Save the Model

In [ ]:
#model.save("custom_image_classifier.h5")

# 10. Predict on a new image

In [ ]:
# def predict_image(img_path):
#     img = image.load_img(img_path, target_size=IMAGE_SIZE)
#     img_array = image.img_to_array(img)
#     img_array = tf.expand_dims(img_array, 0)  # create batch
#     predictions = model.predict(img_array)
#     predicted_class = class_names[np.argmax(predictions)]
#     confidence = np.max(predictions)
#     print(f"Predicted: {predicted_class} ({confidence*100:.2f}%)")

# Example usage

In [ ]:
# predict_image("path/to/test.jpg")